In [ ]:
import numpy as np
from numpy.random import default_rng
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import distance

from sklearn.preprocessing import StandardScaler
from sklearn import decomposition

from PIL import Image
import networkx as nx

import os

import IBloFunMatch_inter as ibfm

output_dir = "output"

# Read data by class
Read data by class and store into a list

In [ ]:
num_class = 20
num_samples = 72
y = []
for c in range(num_class):
    y += [c]*num_samples

In [ ]:
data = []
for c in range(1, num_class+1):
    for i in range(num_samples):
        im_frame = Image.open(f"coil-20-proc/coil-20-proc/obj{c}__{i}.png")
        np_frame = np.array(im_frame)
        data.append(np_frame.ravel())
    # samples per class
# going through classes
data = np.array(data)

Function to store cycle representatives.

In [ ]:
def draw_repr_cycle(repr_cycle, figsize, data):
    fig, ax = plt.subplots(figsize=figsize)
    G = nx.Graph()
    for v in np.unique(repr_cycle):
        G.add_node(v)
    weighted_edges = []
    for edge in np.array(repr_cycle).reshape((-1,2)).tolist():
        weighted_edges.append((edge[0], edge[1], distance.euclidean(data[edge[0]], data[edge[1]])))
    # G.add_edges_from(weighted_edges)
    G.add_weighted_edges_from(weighted_edges)
    pos = nx.spectral_layout(G)
    nx.draw_networkx(G, ax=ax, pos=pos, width=figsize[0])
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    for node in pos.keys():
        im_array = data[node].reshape((128,128))
        im_frame = Image.fromarray(im_array)
        center = origin=pos[node]
        extent = (center[0]-0.03, center[0]+0.03, center[1]-0.03, center[1]+0.03)
        ax.imshow(im_array, cmap="gray", extent=extent, zorder=4)
    
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)
    return ax

# Perform matchings per each class

In [ ]:
DATA_PERCENT = 0.3

Create directories to store the diagrams of matched classes.

In [ ]:
for i in range(20):
    os.makedirs(f"plots/COIL_CLASS/class_{i}", exist_ok=True)

In [ ]:
rng = default_rng(22)
for cidx in range(20):
    # Take class data
    class_indices = np.nonzero(np.array(y)==cidx)[0].tolist()
    class_data = data[class_indices]
    # Take a very well chosen subset
    # subset_idx_good = np.linspace(0, len(class_indices), int(len(class_indices)*DATA_PERCENT)).astype(int)[:-1].tolist()
    # subset_idx_good = np.array(class_indices)[subset_idx_good].tolist()
    # exp_indices = [subset_idx_good]
    exp_indices= []
    # Take 10 subsets randomly
    for i in range(30):
        exp_indices.append(np.sort(rng.choice(class_indices, int(len(class_indices)*DATA_PERCENT), replace=False)).tolist()) 
    exp_match = []
    for id_exp in range(len(exp_indices)):
        indices_subset = [class_indices.index(i) for i in exp_indices[id_exp]]
        exp_indices.append(indices_subset) 
        exp_match.append(ibfm.get_IBloFunMatch_subset(None, class_data, indices_subset, output_dir, num_it=4, points=True, max_rad=-1))
    # finished with matchings
    # Plot matchings
    for id_exp, ibfm_out in enumerate(exp_match):
        fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(8,4))
        if(ibfm_out["S_barcode_1"].shape[0]>0):
            ibfm.plot_matching(ibfm_out, output_dir, ax, fig, dim=1, frame_on=True, strengths=False)
        plt.savefig(f"plots/COIL_CLASS/class_{cidx}/matching_1_{id_exp}.png")
    # Write max and sum scores
    max_scores, sum_scores = [], []
    for id_exp, ibfm_out in enumerate(exp_match):
        if(ibfm_out["S_barcode_1"].shape[0]>0):
            max_scores.append(np.max(ibfm_out["matching_strengths_1"]))
            sum_scores.append(np.sum(ibfm_out["matching_strengths_1"]))
        else:
            max_scores.append(0)
            sum_scores.append(0)
    # end adding scores 
    np.savetxt(f"plots/COIL_CLASS/class_{cidx}/max_scores.txt", max_scores, fmt='%4.4f')
    np.savetxt(f"plots/COIL_CLASS/class_{cidx}/sum_scores.txt", sum_scores, fmt='%4.4f')

In [ ]:
%%capture
figsizes=[(4,2), (8,4)]
for id_exp, ibfm_out in enumerate(exp_match):
    fig, ax = plt.subplots(ncols=2, nrows=1, figsize=figsizes[1])
    if(ibfm_out["S_barcode_1"].shape[0]>0):
        ibfm.plot_matching(ibfm_out, output_dir, ax, fig, dim=1, frame_on=True, strengths=False)
    plt.savefig(f"plots/COIL_class/self_matching_exp_{id_exp}.png")

In [ ]:
print(exp_indices)

# Plot Cycles for illustration

Plot cycle in codomain.

In [ ]:
%%capture
repr_cycle = exp_match[0]["X_reps_1"][-1]
figsize = (len(repr_cycle), len(repr_cycle))
ax = draw_repr_cycle(repr_cycle, figsize, class_data)
plt.savefig(f"plots/COIL_CLASS/cycle_rep_codomain.png")

In [ ]:
exp_match[0].keys()

In [ ]:
%%capture
for id_exp, ibfm_out in enumerate(exp_match):
    if ibfm_out["S_barcode_1"].shape[0]==0:
        continue
    repr_cycle = ibfm_out["S_reps_1"][ibfm_out["matching_strengths_1"].argmax()]
    figsize = (len(repr_cycle), len(repr_cycle))
    ax = draw_repr_cycle(repr_cycle, figsize, class_data)
    strength = np.max(ibfm_out["matching_strengths_1"])
    ax.set_title(f"Matching Strength: {strength}", fontsize=3*figsize[0], c="yellow")
    plt.savefig(f"plots/COIL_CLASS/cycle_rep_{id_exp}.png")

Plot cycles from best experiment. This is number 0.

In [ ]:
%%capture 
for idx_cycle, repr_cycle in enumerate(exp_match[0]["S_reps_1"]):
    figsize = (len(repr_cycle), len(repr_cycle))
    ax = draw_repr_cycle(repr_cycle, figsize, class_data)
    plt.savefig(f"plots/COIL_CLASS/cycle_rep_good_{idx_cycle}.png") 

Plot cycles from worst.

In [ ]:
%%capture 
for idx_cycle, repr_cycle in enumerate(exp_match[8]["S_reps_1"]):
    figsize = (len(repr_cycle), len(repr_cycle))
    ax = draw_repr_cycle(repr_cycle, figsize, class_data)
    plt.savefig(f"plots/COIL_CLASS/cycle_rep_worst_{idx_cycle}.png") 

In [ ]:
exp_match[8]["S_barcode_1"]

In [ ]:
exp_match[0]["S_barcode_1"]